In [5]:
import openai
import re
import httpx
import os 
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
from openai import OpenAI

In [6]:
client = OpenAI()

In [7]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello World"}],
)

In [8]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [9]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute() # This will be defined below
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4-0125-preview",
            temperature=0,
            messages=self.messages
        )
        return completion.choices[0].message.content

In [10]:
prompt = """AgentYou run in a loop of Thought, Action, Pause, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then retun PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed


Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weighs 103 pounds

You then output:

Answer: Bulldogs are about 103 pounds
""".strip()

In [11]:
def calculate(math_expression: str):
    return eval(math_expression)

def average_dog_weight(name: str):
    if name in "Scottish Terrier":
        return ("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return ("A Border Collie average weight is 37 lbs")
    elif name in "Toy Poodle":
        return ("A Toy Poodle average weight is 7 lbs")
    else:
        return ("An average dog weight is 50 lbs")
    

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [13]:
abot = Agent(prompt)

In [14]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the dog's weight using average_dog_weight for a Toy Poodle.
Action: average_dog_weight: Toy Poodle
PAUSE


In [15]:
result = average_dog_weight("Toy Poodle")
print(result)

A Toy Poodle average weight is 7 lbs


In [16]:
next_prompt = f"Observation: {result}"

In [17]:
abot(next_prompt)

'Answer: Toy Poodles weigh about 7 pounds.'

In [18]:
abot.messages

[{'role': 'system',
  'content': 'AgentYou run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then retun PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weighs 103 pounds\n\nYou then output:\n\nAnswer: Bulldogs are about 103 pounds'},
 {'role': 'user', 'content': 'H

In [19]:
abot = Agent(prompt)

In [20]:
question = """I have 2 dogs, a border collie and a scottish terrier. 
I want to know what is their combined weight.""" 

In [21]:
abot(question)

'Thought: To find the combined weight of the two dogs, I need to find the average weight of a Border Collie and a Scottish Terrier separately and then add those weights together.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [23]:
next_prompt = f"Observation: {average_dog_weight('Border Collie')}"
print(next_prompt)

Observation: A Border Collie average weight is 37 lbs


In [24]:
abot(next_prompt)

'Thought: Now that I know the average weight of a Border Collie is 37 lbs, I need to find the average weight of a Scottish Terrier to calculate the combined weight.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [25]:
next_prompt = f"Observation: {average_dog_weight('Scottish Terrier')}"
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [26]:
abot(next_prompt)

'Thought: With the average weight of a Border Collie being 37 lbs and a Scottish Terrier being 20 lbs, I can now calculate their combined weight.\nAction: calculate: 37 + 20\nPAUSE'

In [29]:
next_prompt = f"Observation: {calculate('37 + 20')}"
print(next_prompt)

Observation: 57


In [30]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is about 57 pounds.'

In [31]:
abot.messages

[{'role': 'system',
  'content': 'AgentYou run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then retun PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weighs 103 pounds\n\nYou then output:\n\nAnswer: Bulldogs are about 103 pounds'},
 {'role': 'user',
  'content': 